In [1]:
from __future__ import unicode_literals

In [2]:
import sys
sys.path.append('Perfomer_local/keras_layer_normalization')
sys.path.append('Perfomer_local/keras_performer')
sys.path.append('Perfomer_local/keras_position_wise_feed_forward')
sys.path.append('Perfomer_local/tensorflow_fast_attention')

'''
sys.path.append('D:\Code\Project-AI-JAVA-ANNOTATION-2021\Performer\Perfomer_local\keras_layer_normalization')
sys.path.append('D:\Code\Project-AI-JAVA-ANNOTATION-2021\Performer\Perfomer_local\keras_performer')
sys.path.append('D:\Code\Project-AI-JAVA-ANNOTATION-2021\Performer\Perfomer_local\keras_position_wise_feed_forward')
sys.path.append('D:\Code\Project-AI-JAVA-ANNOTATION-2021\Performer\Perfomer_local\tensorflow_fast_attention')
'''

"\nsys.path.append('D:\\Code\\Project-AI-JAVA-ANNOTATION-2021\\Performer\\Perfomer_local\\keras_layer_normalization')\nsys.path.append('D:\\Code\\Project-AI-JAVA-ANNOTATION-2021\\Performer\\Perfomer_local\\keras_performer')\nsys.path.append('D:\\Code\\Project-AI-JAVA-ANNOTATION-2021\\Performer\\Perfomer_local\\keras_position_wise_feed_forward')\nsys.path.append('D:\\Code\\Project-AI-JAVA-ANNOTATION-2021\\Performer\\Perfomer_local\tensorflow_fast_attention')\n"

In [3]:
from sklearn.model_selection import train_test_split
import unittest
import numpy as np
from keras_performer import performer as tfr
import nltk

from os import listdir
from os.path import isfile, isdir, join
from nltk.tokenize import wordpunct_tokenize

nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\W.R_Chen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
'''
def solve_cudnn_error():
    import tensorflow as tf
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)
            
solve_cudnn_error()
'''

'\ndef solve_cudnn_error():\n    import tensorflow as tf\n    gpus = tf.config.experimental.list_physical_devices(\'GPU\')\n    if gpus:\n        try:\n            # Currently, memory growth needs to be the same across GPUs\n            for gpu in gpus:\n                tf.config.experimental.set_memory_growth(gpu, True)\n            logical_gpus = tf.config.experimental.list_logical_devices(\'GPU\')\n            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")\n        except RuntimeError as e:\n            # Memory growth must be set before GPUs have been initialized\n            print(e)\n            \nsolve_cudnn_error()\n'

In [5]:
import csv
def readCSV(file_name):
  errlist=[]
  LBlist=[]
  with open(file_name, newline='') as csvfile:

  # 讀取 CSV 檔內容，將每一列轉成一個 dictionary

    rows = csv.DictReader(csvfile)
    for row in rows: 
      RL=list(row.values())

      RL = list(map(int, RL))
      errs=RL[1:37]
      LB=RL[37:]
      errlist.append(errs)
      LBlist.append(LB)
  return errlist,LBlist


In [6]:
def find_first_sublist(seq, sublist, start=0):
    length = len(sublist)
    for index in range(start, len(seq)):
        if seq[index:index+length] == sublist:
            return index, index+length

In [7]:
def replace_sublist(seq, sublist, replacement):
    length = len(replacement)
    index = 0
    for start, end in iter(lambda: find_first_sublist(seq, sublist, index), None):
        seq[start:end] = replacement
        index = start + length

In [8]:
def replaceTAGS(x):
    replace_sublist(x, ['<', 'BOC', '>'], ["<BOC>"])
    replace_sublist(x, ['<', 'EOC', '>'], ["<EOC>"])
    replace_sublist(x, ['<', 'BOTM', '>'], ["<BOTM>"])
    replace_sublist(x, ['<', 'BOT', '>'], ["<BOT>"])
    replace_sublist(x, ['<', 'EOT', '>'], ["<EOT>"])
    replace_sublist(x, ['<', 'BOM', '>'], ["<BOM>"])
    replace_sublist(x, ['<', 'EOM', '>'], ["<EOM>"])
    replace_sublist(x, ['<', 'EOTM', '>'], ["<EOTM>"])
    replace_sublist(x, ['<', 'CR', '>'], ["<CR>"])
    return x

In [9]:
def parseSentence(x):	
	tokenizer = RegexpTokenizer(r"[\w']+|[].,:!?;=+-\\*/@#$%^&_(){}~|\"[]")
	tokens=[]
	state="START"
	chrs=""
	for i in range(len(x)):
		#print(ord(x[i]))
		if (ord(x[i])>255):
			inp="U"
		else:
			inp="E"
	
		if state=="START":
			if inp=="E":
				state="ASCII"
				chrs=x[i]
			else:
				state="UNICODE"
				tokens.append(x[i])
			
		elif state=="ASCII":
			if inp=="E":
				chrs += x[i]
			else:#U
				state="UNICODE"
				tokens += tokenizer.tokenize(chrs) #wordpunct_tokenize(chrs)  #nltk.word_tokenize(chrs)
				chrs=""
				tokens.append(x[i])
	
		elif state=="UNICODE":
			if inp=="E":
				state="ASCII"
				chrs=x[i]
			else:
				state="UNICODE"
				tokens.append(x[i])
	if len(chrs)>0:
		tokens += tokenizer.tokenize(chrs) #wordpunct_tokenize(chrs)  # nltk.word_tokenize(chrs) 
	return tokens

In [10]:
def readcode(fname):
    with open(fname) as f:
        data = f.read()
        return data

In [11]:
class TestTranslate(unittest.TestCase):
 
    @staticmethod
    def _build_token_dict(token_list):
        token_dict = {
            '<PAD>': 0,
            '<START>': 1,
            '<END>': 2,
            '<BOC>': 3,
            '<EOC>': 4,
            '<BOTM>': 5,
            '<BOT>': 6,
            '<EOT>': 7,
            '<BOM>': 8,
            '<EOM>': 9,
            '<EOTM>': 10,
            '<CR>': 11,
        }
        for tokens in token_list:
            for token in tokens:
                if token not in token_dict:
                    token_dict[token] = len(token_dict)
        return token_dict
 
    def test_translate(self):
        print("i am here: " )
        source_tokens = []
        target_errors=[]
        target_LB=[]
        #source_file=[]
        
        Input_Path = "D:\Code\Project-AI-JAVA-ANNOTATION-2021\Performer\Perfomer_local\Trianing\InputTxt"
        Location_Output_Path = "D:\Code\Project-AI-JAVA-ANNOTATION-2021\Performer\Perfomer_local\Trianing\InputCSV"
        '''
        Input_Path = "Performer/Perfomer_local/Trianing/InputTxt"
        Location_Output_Path = "Performer/Perfomer_local/Trianing/InputCSV"
        '''
 
        InputFiles = sorted(listdir(Input_Path))
        OutputFiles = sorted(listdir(Location_Output_Path))
        #max_files =10
        for f in InputFiles:
          fullpath = join(Input_Path,f)
          #source_file=fullpath
          if isfile(fullpath):
            source_tokens.append(parseSentence(readcode(fullpath)))
            #if len(source_tokens)>max_files: break
        for f in OutputFiles:
          fullpath = join(Location_Output_Path,f)
          #source_file=fullpath
          if isfile(fullpath):
            err,lb = readCSV(fullpath)
            target_errors.append(err)
            target_LB.append(lb)
            #if len(source_tokens)>max_files: break
        dd=np.asarray(target_errors)
        print("AAAA: ", dd.shape)
        print("aaaa: " , type(target_errors[0][0]))
        target_errors=target_errors[0]  
        target_LB=target_LB[0]     
        print("XXXX: " , len(source_tokens))
        print("YYYY: " , type(target_errors[0][0]))
        print("ZZZZ: " , len(target_LB))

 
        source_tokens2 = []
        target_errors2= []
        target_LB2=[]

        THRESHOLD_FILE_LEN = 64000000
 
        for i in range(len(source_tokens)): 
          #print (i)
          src = source_tokens[i]
          err = target_errors[i]   
          lb = target_LB[i]                   
          if (len(src)<=THRESHOLD_FILE_LEN):
            source_tokens2.append(src)
            target_errors2.append(err)
            target_LB2.append(lb) 
        source_tokens = source_tokens2
        target_errors= target_errors2        
        target_LB= target_LB2        

        print("XXXX2: " , len(source_tokens))
                    
 
        # Generate dictionaries
        source_token_dict = self._build_token_dict(source_tokens)
        
 
        # Add special tokens
        encode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in source_tokens]
        
        #output_tokens = [tokens + ['<END>', '<PAD>'] for tokens in target_tokens]
 
        # Padding
        self.sl = max(map(len, encode_tokens))
        
        source_max_len = self.sl
        
 
        encode_tokens = [tokens + ['<PAD>'] * (source_max_len - len(tokens)) for tokens in encode_tokens]
        
        
 
        encode_input = [list(map(lambda x: source_token_dict[x], tokens)) for tokens in encode_tokens]
       
        
        #print("encode_input:", encode_input)
        
        #print("len(source_token_dict):", len(source_token_dict))
        
        
        #decode_output_one_hot_encoded = to_categorical(decode_output)
        #print("one-hot-decode_output:", decode_output_one_hot_encoded)
        #decode_output =  list(decode_output_one_hot_encoded)
        token_num =len(source_token_dict)
        print(type(token_num))
        #Build & fit model
        #Set model para    
        model = tfr.get_model(
            max_input_len= (source_max_len),
            errNum=36,
            lbNum=192,
            token_num=len(source_token_dict),
            embed_dim=32, #32,
            encoder_num=4, #2 max = 6
            head_num=2,#4
            hidden_dim=64, #128
            dropout_rate=0.05 #0.05
        )
        
        losses = {    
            "error_feed_forward_output1": "categorical_crossentropy",
            "error_feed_forward_output2": "mean_squared_error",            
        }
        
        lossWeights = {"error_feed_forward_output1": 1.0,"error_feed_forward_output2": 1.0}
        
        #opt = adam(learning_rate=0.00001) #can change learning rate more lower default rate = 0.001
        
        #model.compile(optimizer=opt, loss=losses, loss_weights=lossWeights)
        
        model.compile('adam', loss=losses, loss_weights=lossWeights)
        
        print("x.shape", np.asarray(encode_input).shape)  #x.shape (2,  9)
        
 
        #x=[np.array(encode_input * 1)]
        #y=[np.array(target_errors * 1),np.array(target_LB * 1)]
       
        #print("x.shape", np.asarray(x).shape)  #x.shape (2, 2048, 9)
        
        ####  Split the data set into train and test_model
        x=np.array(encode_input)
        y=list(zip(np.array(target_errors), np.array(target_LB)))


        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

        y_train = list(zip(*y_train))
        y_train[0] = np.array(y_train[0])
        y_train[1] = np.array(y_train[1])

        
        y_test = list(zip(*y_test))
        y_test[0] = np.array(y_test[0])
        y_test[1] = np.array(y_test[1])
               
        
        saveTestTrainData("train_models/x_train_500.npy", x_train)
        saveTestTrainData("test_models/x_test_500.npy", x_test)
        saveTestTrainData("train_models/y_train0_500.npy", y_train[0])#target
        saveTestTrainData("train_models/y_train1_500.npy", y_train[1])#lb
        saveTestTrainData("test_models/y_test0_500.npy", y_test[0])
        saveTestTrainData("test_models/y_test1_500.npy", y_test[1])
    
        ####
        #if you only need to train model don't use this
        
        model.fit(
            x=x_train,
            y=y_train,
            epochs=500, #100 200 500
            batch_size=32,
        )
        
        model.save("test_model1.h5")
        saveDictionary(source_token_dict, 'source_token_dict.pickle')
        
        model, source_token_dict = load("test_model1.h5")

        # Predict
        x_train_loaded = loadTestTrainData("train_models/x_train_500.npy") 
        y_train_loaded = [  loadTestTrainData("train_models/y_train0_500.npy"),
                          loadTestTrainData("train_models/y_train1_500.npy") ]

        x_test_loaded =  loadTestTrainData("test_models/x_test_500.npy") 
        y_test_loaded = [ loadTestTrainData("test_models/y_test0_500.npy"),
                         loadTestTrainData("test_models/y_test1_500.npy")]
        
        out1, out2 = tfr.decode(
            model,
            #encode_input,
            x_test_loaded,max_len=source_max_len
        )
        print("out1: ", out1)
        #print("target_errors: ",target_errors) 
        print("target_errors: ", y_test_loaded[0])        
        print("out2: ", out2)
        #print("target_LB: ", target_LB)
        print("target_LB: ", y_test_loaded[1])
        
    def getsource_max_lan(self):
        return self.sl

In [12]:
def saveDictionary(dt, file):
        import pickle
        a_file = open(file, "wb")
        pickle.dump(dt, a_file)
        a_file.close()

In [13]:
def loadDictionary(file):
        import pickle
        a_file = open(file, "rb")
        dt = pickle.load(a_file)
        return dt

In [14]:
def saveTestTrainData(filename, data): # e.g., 'test.npy'
    with open(filename, 'wb') as f:
        np.save(f, data)

In [15]:
def loadTestTrainData(filename): # e.g., 'test.npy'    
    with open(filename, 'rb') as f:
        a = np.load(f)
        return a

In [16]:
def load(model_name):
        import sys
        sys.path.append('Perfomer_local/keras_position_wise_feed_forward')
        sys.path.append('Perfomer_local/tensorflow_fast_attention')
        sys.path.append('Perfomer_local/keras_performer')
        #sys.path.append('/Perfomer_local/keras_pos_embed')

        from keras_performer import performer
        from tensorflow import keras
        from keras_embed_sim import EmbeddingRet, EmbeddingSim
        from keras_pos_embd import TrigPosEmbedding
        from tensorflow_fast_attention.fast_attention import  Attention, SelfAttention
        from keras_position_wise_feed_forward.feed_forward import FeedForward  

        co = performer.get_custom_objects()

        model = keras.models.load_model(model_name, custom_objects= co)
        source_token_dict = loadDictionary('source_token_dict.pickle')
       # t = loadDictionary(target_token_dict, 'target_token_dict.pickle')
       # t_inv = loadDictionary(target_token_dict_inv, 'target_token_dict_inv.pickle')
        return model, source_token_dict,

In [17]:
x=TestTranslate()
x.test_translate()

i am here: 
AAAA:  (1, 359, 36)
aaaa:  <class 'list'>
XXXX:  359
YYYY:  <class 'int'>
ZZZZ:  359
XXXX2:  359
<class 'int'>
In get_model: encoder_input:  (None, 2919)
The following Variables were used a Lambda layer's call (tf.compat.v1.nn.embedding_lookup), but
are not present in its tracked objects:
  <tf.Variable 'Token-Embedding/embeddings:0' shape=(1463, 32) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
In get_model: encoder_embed:  (None, 2919, 32)
Start Warpping................................
OOOO: KerasTensor(type_spec=TensorSpec(shape=(None, 2919, 32), dtype=tf.float32, name=None), name='tf.__operators__.add_1/AddV2:0', description="created by layer 'tf.__operators__.add_1'")
embed_dim: 32
head_num: 2
dropout_rate: 0.05
masked: False
query_input: XXXXXXX  (None, 2919, 32)
source_input: XXXXXXX  (None, 2919, 32)
Qu

TypeError: Cannot convert a symbolic Keras input/output to a numpy array. This error may indicate that you're trying to pass a symbolic value to a NumPy call, which is not supported. Or, you may be trying to pass Keras symbolic inputs/outputs to a TF API that does not register dispatching, preventing Keras from automatically converting the API call to a lambda layer in the Functional Model.

In [ ]:
# the following funcs is make socre to model 
testmodel = "test_model1.h5"

In [ ]:
def loadTestdata():
    import numpy as np
    #load model and dic ps. dic is not use
    model, source_token_dict = load(testmodel)
    #load
    x_test_loaded = loadTestTrainData("test_models/x_test_500.npy") 
    y_test_loaded = [ loadTestTrainData("test_models/y_test0_500.npy"), loadTestTrainData("test_models/y_test1_500.npy")] 
    out1, out2 = tfr.decode(model,x_test_loaded,max_len=x.getsource_max_lan()) 
    #print((y_test_loaded[0][1])) #Error_type #vs out1
    #print((y_test_loaded[1][1])) #Line_Block #vs out2
    #print((out1[1]))#prob upper then 0.5
    #print((out2[1]))#prob upper then 0.5
    #print(int(out2))#prob lb
    test_ep = np.around(out1)
    test_lb = np.around(out2)
    ans_ep = np.around(y_test_loaded[0])
    ans_lb = np.around(y_test_loaded[1])
    print(test_ep[1])
    print(test_lb[1])
    print(ans_ep[1])
    print(ans_lb[1])
    #scoreTest(test_ep,test_lb,ans_ep,ans_lb) #go to test the score

In [ ]:
def scoreTest(test_ep,test_lb,ans_ep,ans_lb):
    
    for i in range()

In [ ]:
loadTestdata()